<a href="https://colab.research.google.com/github/sowmyarajesh/ML-NLP/blob/main/NLP_EmotionDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Dataset

Get the original emotions data from https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp

copy of it is provided in the git repo too https://github.com/sowmyarajesh/ML-NLP

This dataset has a sentence and its associated emotion name seperated by ;.




In [ ]:
trainData = pd.read_csv('train.txt', sep=';', header=None)
valData = pd.read_csv('val.txt', sep=';', header=None)
testData = pd.read_csv('test.txt', sep=';', header=None)
trainData.columns

Int64Index([0, 1], dtype='int64')

In [ ]:
trainData[1].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: 1, dtype: int64

###  Preprocessing

In [ ]:
padding_type='post'
oov_label ="<oov>"

In [ ]:
tr_sentences = trainData[0].values
tr_labels = trainData[1].values

val_sentences = valData[0].values
val_labels = valData[1].values

test_sentences = testData[0].values
test_labels = testData[1].values

In [ ]:
# X
tokenizer  = Tokenizer(oov_token=oov_label)
tokenizer.fit_on_texts(tr_sentences)
XIndex = tokenizer.word_index
train_seq = tokenizer.texts_to_sequences(tr_sentences)
train_X = np.array(pad_sequences(train_seq,padding=padding_type))
print("creating X from sentences")
print("padded sequence[0] = ",train_X[0])
print("padded sequence-> X.shape: ",train_X.shape)
print("Length of input index: ",len(XIndex.keys()))

#y
tokenizerY  = Tokenizer(oov_token=oov_label)
tokenizerY.fit_on_texts(tr_labels)
yIndex = tokenizerY.word_index
train_Labels = np.array([y[0] for y in tokenizerY.texts_to_sequences(tr_labels)])
print(yIndex)
print(train_Labels[:10])

[  2 139   3 679   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
(16000, 66)
15213
{'<oov>': 1, 'joy': 2, 'sadness': 3, 'anger': 4, 'fear': 5, 'love': 6, 'surprise': 7}
[3 3 4 6 4 3 7 5 2 6]


we have 16000 trainig sequences with 66 tokens. Now convert the text to sequence with the same training tokens for validation and test data

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_sentences)
test_padded = np.array(pad_sequences(test_seq,padding=padding_type))
test_y = np.array([y[0] for y in tokenizerY.texts_to_sequences(test_labels)])
print(test_padded[0])
print(test_padded.shape)
test_y[:10]

[  17    8  203  715   15   17   26   46 5595  114   58    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
(2000, 61)


array([3, 3, 3, 2, 3, 5, 4, 2, 2, 4])

In [ ]:
val_seq = tokenizer.texts_to_sequences(val_sentences)
val_padded = np.array(pad_sequences(val_seq,padding=padding_type))
val_y = np.array([y[0] for y in tokenizerY.texts_to_sequences(val_labels)])
print(val_padded[0])
print(val_padded.shape)

[   17     8   157   260     4   343    16    51    19   212 11289    50
    10    13   533     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0]
(2000, 61)


### Training/ validation

In [ ]:
vocab_size = len(XIndex.keys())+1
embedding_dim = len(yIndex.keys())+1
max_length = tr_padded.shape[1]  # number of columns in the input matrix
epochs = 30

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])
model.compile( optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)

In [ ]:
history = model.fit(tr_padded, train_Labels, epochs=epochs, validation_data=(val_padded, val_y), verbose=2)

Epoch 1/30


ValueError: ignored